# Getting started
[Link to tutorial](https://octopus-code.org/documentation/13/tutorial/basics/getting_started/)

The objective of this tutorial is to give a basic idea of how **Octopus** works.

In [ ]:
!mkdir -p 1-getting_started

In [ ]:
cd 1-getting_started

## Generating the input file
With the [`%%writefile` magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile), create a text file called `inp` containing the following text:

In [ ]:
%%writefile inp

stdout = "stdout_gs.txt"
stderr = "stderr_gs.txt"

CalculationMode = gs

%Coordinates
 'H' | 0 | 0 | 0
%
Spacing = 0.25 * angstrom
Radius = 4.0 * angstrom

This is the simplest example of an **Octopus** input file:

* [`CalculationMode = gs`](https://octopus-code.org/documentation//13/variables/calculation_modes/calculationmode): This variable defines the run mode -- please consult the manual for the full list of the possible run modes. In this case we set it to `gs`, which instructs the code to start a ground-state calculation.

* [`%Coordinates`](https://octopus-code.org/documentation//13/variables/system/coordinates/coordinates): The entry is not just the definition of a variable, but rather of a full set of them -- a "block" of variables. The beginning of a block is marked by the `%identifier` line, and ended by a `%` line. In this case the identifier is [`Coordinates`](https://octopus-code.org/documentation//13/variables/system/coordinates/coordinates), where we list the atoms or species in our calculation and its coordinates, one per line. In this case, we put a single hydrogen atom in the center of our simulation box. 

The reason this input file can be so simple is that **Octopus** comes with default values for the simulation parameters, and a set of default pseudopotentials for several elements (for properly converged calculations you might need to adjust these parameters, though).

To get a general idea of the format of the **Octopus** input file, go and read the page about the [Input file](https://octopus-code.org/documentation//13/manual/basics/input_file) in the manual.

The documentation for each input variable can be found in the [variable reference](https://octopus-code.org/documentation//13/variables/) online, and can also be accessed via the [oct-help](https://octopus-code.org/documentation//13/manual/external_utilities/oct-help) utility.


## Running Octopus
Once you have written your input file, run the octopus command (using mpirun and perhaps a job script if you are using the parallel version). If everything goes correctly, you should see several lines of output in the terminal (if you don’t, there must be a problem with your installation). As this is probably the first time you run Octopus, we will examine the most important parts of the output.

In [ ]:
!octopus # Run octopus with the  previously defined input file and log the output of stdout and stderr

* First there is an octopus drawn in ASCII art, the copyright notice and some information about the octopus version you are using and the system where you are running:


In [ ]:
!head -n 55 stdout_gs.txt

Note that it also gives you the revision number, the compiler, and the compiler flags used. You should always include this information when submitting a bug report!

* The type of calculation it was asked to perform:

In [ ]:
!grep -am 1 -A 3 "Calculation Mode" stdout_gs.txt

* The species and pseudopotentials it is using:

In [ ]:
!grep -am 1 -A 17 "Species" stdout_gs.txt

* After some other output, **Octopus** prints information about the grid: as we didn't say anything in the input file, **Octopus** used the parameters recommended for this pseupopotential:

In [ ]:
!grep -am 1 -A 9 " Grid" stdout_gs.txt

* The level of theory and, in the case of (TD)DFT, the approximation to the exchange-correlation term:

In [ ]:
!grep -am 1 -A 12 "Theory Level" stdout_gs.txt

* At this point, **Octopus** tries to read the wave-functions from a previous calculation. As there are none, it will give a warning.

In [ ]:
!cat stderr_gs.txt

* Now **Octopus** commences the calculation. To get a reasonable starting point for the DFT calculation, the initial wavefunctions are calculated as a [Linear Combination of Atomic Orbitals](https://octopus-code.org/documentation/13/manual/calculations/ground_state/#lcao) (LCAO).


In [ ]:
!grep -am 1 -A 7 "initial LCAO calculation" stdout_gs.txt

* After the LCAO, the real DFT calculation starts. For each self-consistency step some information is printed. When SCF [converges](https://octopus-code.org/documentation/13/manual/calculations/ground_state/#convergence), the calculation is done.

In [ ]:
!grep -am 1 -A 10 "SCF CYCLE" stdout_gs.txt

In [ ]:
!grep -am 1 -B 25 -A 10  "Walltime" stdout_gs.txt

Just running the command `octopus` will write the output directly to the terminal. To have a saved copy of the output, it is generally advisable to redirect the output into a file, and to capture the standard error stream as well, which can be specifying `stdout = stdout.txt` and `stderr = stderr.txt`.
That would create a file called `stdout.txt` containing all output and a file called `stdout.txt` containing all warnings and errors.

## Analyzing the results
After finishing the calculation you will find a series of files in the directory you ran:

In [ ]:
!ls

For the moment we will ignore the '''exec'''  and  '''restart''' directories and focus on the `static/info` file, which contains the detailed results of the ground-state calculation. If you open that file, first you will see some parameters of the calculations (that we already got from the output) and then the calculated energies and eigenvalues in Hartrees:


In [ ]:
!cat static/info

Since by default **Octopus** does a spin-unpolarized density-functional-theory calculation with the local-density approximation, our results differ from the exact total energy of 0.5 H. Our exchange-correlation functional can be set by the variable {{< variable "XCFunctional" >}}, using the set provided by the [libxc](https://www.tddft.org/programs/libxc/) library.


## Extras
If you want to improve the LDA results, you can try to repeat the calculation with spin-polarization:

In [ ]:
%%writefile inp

stdout = "stdout_gs_spin_polarized.txt"
stderr = "stderr_gs_spin_polarized.txt"

CalculationMode = gs

%Coordinates
 'H' | 0 | 0 | 0
%
Spacing = 0.25 * angstrom
Radius = 4.0 * angstrom

SpinComponents = spin_polarized

In [ ]:
!octopus

In [ ]:
!cat stdout_gs_spin_polarized.txt

And if you want to obtain the exact Schödinger equation result (something possible only for very simple systems like this one) you have to remove the self-interaction error (a problem of the LDA). Since we only have one electron the simplest way to do it for this case is to use independent electrons:


In [ ]:
%%writefile inp

stdout = "stdout_gs_spin_polarized_independent.txt"
stderr = "stderr_gs_spin_polarized_independent.txt"

CalculationMode = gs

%Coordinates
 'H' | 0 | 0 | 0
%
Spacing = 0.25 * angstrom
Radius = 4.0 * angstrom

SpinComponents = spin_polarized

TheoryLevel = independent_particles

In [ ]:
!octopus

In [ ]:
!cat stdout_gs_spin_polarized_independent.txt

A more general way would be to include self-interaction correction.

[Go to *2-Basic_input_options.ipynb*](2-Basic_input_options.ipynb)